##  Классификация изображений
#####  Определение модели авто по фото. 
#####  На основе baseline и данных в нём рекомендаций повысить точность модели.

In [ ]:
# проверяем что видеокарта подключена
!nvidia-smi -L

In [ ]:
# Загружаем обвязку под keras для использования продвинутых библиотек аугментации
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor update tensorflow
    # Загрузка модели efficientnet
!pip install -q efficientnet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import *
from sklearn.model_selection import train_test_split, StratifiedKFold
from ImageDataAugmentor.image_data_augmentor import *
from keras.models import load_model
import albumentations
import efficientnet.tfkeras as efn

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

# смотрим входные данные и версии 
print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

In [ ]:
# Для вывода кривых training_acc и vol_accuracy,  training_loss и vol_loss
def learning_graphic(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
 
    epochs = range(len(acc))
 
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
 
    plt.figure()
 
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
    return plt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 8  # эпох на обучение
BATCH_SIZE           = 32 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-3
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')
sample_submission = pd.read_csv(DATA_PATH + "sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH + data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH + f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index + 1)
    plt.imshow(im)
    plt.title('Class: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Изображения имеют различный формат и соотношение сторон. <br>
Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

* Для классификации представлены фотографии 10 категорий авто
* Категории сбалансированны по кол-ву изображений
* 15561 фото в трейне. Так как изображений в трейне не очень много, лучше использовать аугментации для увеличения обучающей выборки.

# Подготовка данных

### Аугментация данных

Аугментация данных очень важна когда у нас не большой датасет (как в нашем случае)

In [ ]:
AUGMENTATIONS = albumentations.Compose([
    #  Увеличим вероятности сдвига, масштабирования, поворота, т.к. по умолчанию их значения низкие. 
    albumentations.ShiftScaleRotate(shift_limit=0.0625, 
                       scale_limit=0.01, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=20, 
                       p=.75), 
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

In [ ]:
# Подготовка для генерации данных:
train_datagen = ImageDataAugmentor(
        rescale=1/255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )
        
test_datagen = ImageDataAugmentor(rescale=1/255)

### Генерация данных

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH + 'train/',      
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset = 'training') 

test_generator = train_datagen.flow_from_directory(
    PATH + 'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset = 'validation') 

In [ ]:
# посмотрим результаты
train_generator.show_data(rows=3, cols=5)

# Построение модели

### Загружаем предобученную сеть FixEfficientNet-B7

In [ ]:
base_model = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
base_model.summary()

In [ ]:
# Для начала заморозим веса EfficientNetB7 и обучим только "голову". 
# Делаем это для того, чтобы хорошо обученные признаки на Imagenet не затирались в самом начале нашего обучения
base_model.trainable = False

In [ ]:
import tensorflow.keras.layers as Layer
import tensorflow.keras.models as Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
# Устанавливаем новую "голову"
model=Model.Sequential()
model.add(base_model)
model.add(Layer.GlobalAveragePooling2D()) # объединяем все признаки в единый вектор 

# Экспериментируем с архитектурой
# Добавляем ещё один полносвязный слой, dropout и Batch Normalization
model.add(Layer.Dense(256, 
                      activation='relu', 
                      bias_regularizer=l2(1e-4),
                      activity_regularizer=l2(1e-5)))
model.add(Layer.BatchNormalization())
model.add(Layer.Dropout(0.25))
model.add(Layer.Dense(CLASS_NUM, activation='softmax'))


In [ ]:
model.summary()

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [ ]:
# добавим колбэк для сохранения модели с лучшими показателями
checkpoint = ModelCheckpoint('best_model.hdf5', monitor = ['val_accuracy'], verbose = 1, mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, min_lr=0.0000001, verbose=1, mode='auto')
callbacks_list = [checkpoint, earlystop]

Обучаем:

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=LR),
              metrics=['accuracy'])

In [ ]:
# Обучаем

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики обучения:

In [ ]:
learning_graphic(history).show()

### FineTuning. Обучение половины весов

In [ ]:
# проверим количеcтво слоев в базовой модели
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
base_model.trainable = True

fine_tune_at = len(base_model.layers) // 2

# остальные слои также пока заморозим 
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
# проверим по количеству слоев 
len(base_model.trainable_variables)

In [ ]:
BATCH_SIZE = 16 # уменьшим этот показатель
LR = 1e-4 # поэкспериментируем с learning_rate

train_generator = train_datagen.flow_from_directory(
    PATH + 'train/',      
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset = 'training') 

test_generator = train_datagen.flow_from_directory(
    PATH + 'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset = 'validation') 

In [ ]:
# обучаем
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=LR),
              metrics=['accuracy'])

In [ ]:
# добавим колбэк для сохранения модели с лучшими показателями
checkpoint = ModelCheckpoint('best_model2.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
# Обучаем

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model2.hdf5')
model.load_weights('best_model2.hdf5')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила 94%. 
Учитывая что классов 10 - это Очень хороший результат!     
Посмотрим графики обучения:
    

In [ ]:
learning_graphic(history).show()

### FineTuning. Разморозим и дообучим всю сеть EfficientNetB7

In [ ]:
base_model.trainable = True

In [ ]:
BATCH_SIZE = 8 # уменьшим этот показатель
LR = 1e-5 # поэкспериментируем с learning_rate
#IMG_SIZE = 512 # увеличим размер изображений

train_generator = train_datagen.flow_from_directory(
    PATH + 'train/',      
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset = 'training') 

test_generator = train_datagen.flow_from_directory(
    PATH + 'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset = 'validation') 


In [ ]:
# обучаем
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=LR),
              metrics=['accuracy'])

In [ ]:
# добавим колбэк для сохранения модели с лучшими показателями
checkpoint = ModelCheckpoint('best_model_full.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
# Обучаем

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [ ]:
# Сохраним модель
model.save('../working/model_full.hdf5')
model.load_weights('best_model_full.hdf5')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
learning_graphic(history).show()

Модель можно усиливать дальше, так как ещё не началось переобучение.
Сделать это можно за счёт постепенной разморозки слоёв, снижение LR, увеличения числа эпох и т.п

### Модель EfficientNet-B3

In [ ]:
# проведем еще один эксперимент с другой вариацией предобученной сети ENB
# с целью дальнейшего ассамблирования
base_model_2 = efn.EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# построим голову на других принципах сочетания слоев
model = Model.Sequential()
model.add(base_model_2)
model.add(Layer.GlobalAveragePooling2D())
model.add(Layer.BatchNormalization())
model.add(Layer.Dropout(0.25))
model.add(Layer.Dense(256, 
                      activation='relu'))
model.add(Layer.BatchNormalization())
model.add(Layer.Dense(CLASS_NUM, activation='softmax'))

In [ ]:
# применим управляемый learning rate в этой модели
from tensorflow.keras.optimizers.schedules import *
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=ExponentialDecay(
                  0.0009, decay_steps=100, decay_rate=0.9)),
              metrics='accuracy')

In [ ]:
# добавим колбэк для сохранения модели с лучшими показателями
checkpoint = ModelCheckpoint('best_model_enb3.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
# некоторые новые настройки для модели с учетом ранее проведенных экспериментов
EPOCHS               = 4 # эксперименты показали, что такого количества эпох будет достаточно
BATCH_SIZE           = 8 # при большем числе комп не справляется 
VAL_SPLIT            = 0.05 # увеличим обучающую выборку за счет валидационной
IMG_SIZE             = 520 # увеличим размер изображений

In [ ]:
# настройки для аугментации
WIDTH_SHIFT_RANGE    = 0.1
HEIGHT_SHIFT_RANGE   = 0.1
HORIZONTAL_FLIP      = True
VERTICAL_FLIP        = False
ROTATION_RANGE       = 10
BRIGHTNES_RANGE      = (0.5, 1.5)
RESCALE              = 1
SHEAR_RANGE          = 0.2
ZOOM_RANGE           = 0.1

In [ ]:
# также изменим модель аугментации и вообще поэкспериметируем с параметрами
train_datagen = ImageDataGenerator(
#     rescale=1/255,
    validation_split=VAL_SPLIT,
    width_shift_range = WIDTH_SHIFT_RANGE,
    height_shift_range = HEIGHT_SHIFT_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    rotation_range=ROTATION_RANGE,
    shear_range=SHEAR_RANGE,
    brightness_range=BRIGHTNES_RANGE,
    zoom_range=ZOOM_RANGE,
    vertical_flip=VERTICAL_FLIP,
)

val_datagen = ImageDataGenerator(
#     rescale=1/255,
    validation_split=VAL_SPLIT,
)

sub_datagen = ImageDataGenerator(
#     rescale=1/255,
    width_shift_range=WIDTH_SHIFT_RANGE, 
    height_shift_range=HEIGHT_SHIFT_RANGE,
    horizontal_flip=HORIZONTAL_FLIP
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH + 'train/',      
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') 

test_generator = train_datagen.flow_from_directory(
    PATH + 'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') 

In [ ]:
#SUB_PATH = DATA_PATH + 'test_upload/'
#sample_submission = pd.read_csv(DATA_PATH + 'sample-submission.csv')

In [ ]:
sub_generator = sub_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH + 'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list)

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Сохраним модель
model.save('../working/model_last.hdf5')
model.load_weights('best_model_enb3.hdf5')

In [ ]:
learning_graphic(history).show()

In [ ]:
SUB_PATH = DATA_PATH + 'test_upload/'
sample_submission = pd.read_csv(DATA_PATH + 'sample-submission.csv')

In [ ]:
# сделаем сабмит
predictions = model.predict(sub_generator, verbose=1).argmax(axis=1)
submission = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions
}, columns=['Id', 'Category'])
submission.to_csv('submission.csv', index=False)

### Test Time Augmentation (TTA)

In [ ]:
# сделаем несколько предсказаний одной и той же картинки
# усредним эти предсказания
# сохраним в сабмит
sub_generator.reset()
predictions_tta = []
for _ in range(EPOCHS):
    predictions_tta.append(model.predict(sub_generator, verbose=1))
    sub_generator.reset()
predictions_tta = np.mean(np.array(predictions_tta), axis=0).argmax(axis=1)
submission_tta = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions_tta
}, columns=['Id', 'Category'])
submission_tta.to_csv('submission_tta.csv', index=False)

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

### Ансамблирование

In [ ]:
model_2 = load_model('model_last.hdf5')

In [ ]:
sub_generator.reset()
predictions_tta_ansemble = []
for _ in range(EPOCHS):
    predictions_tta_ansemble.append(0.6*model.predict(sub_generator, verbose=1) \
                                    + 0.2*model_2.predict(sub_generator, verbose=1))
    sub_generator.reset()
predictions_tta_ansemble = np.mean(np.array(predictions_tta_ansemble), axis=0).argmax(axis=1)
submission_tta_ansemble = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions_tta_ansemble
}, columns=['Id', 'Category'])
submission_tta_ansemble.to_csv('submission_tta_ansemble.csv', index=False)

# Выводы

В ходе работы проведены все стандартные эксперименты по улучшению показателей модели обучения. По сути в данном проекте созданы две независимые модели. Использовались две модели аугментации - классическая во втором варианте и с библиотекой albumentations в первом, с различными настройками. Экспериментальным путем выведено,что наиболее эффективной предобученной моделью является EfficientNet версии 7.

В варианте с седьмой моделью EfficientNet использовались методы transfer learning и fine-tuning. Использовалась ручная регулировка learning rate и batch. В другом варианте применялся метод ExponentialDecay для регулировки learning rate.

В обоих вариантах применялись функции callback. Так же использовалась Batch Normalization. Архитектуры "голов" сознательно применялись разные.

Модели имеют примерно одинаковую точность 95.28% и 95.86%.

К модели EfficientNetB3, поскольку она имела чуть лучшее значение, применялся метод TTA (Test Time Augmentation), что помогло улучшить показатель точности до 96.64%.
В итоге применен метод ансамблирования. Итоговый показатель на соревновании в Каггл 0.96943.